In [1]:
# Import libraries

import sys
import logging
import pandas as pd
import xarray as xr
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.weather import load_weather
from library.network import build_network

logging.basicConfig(level=logging.INFO)

In [2]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    'EUR': 11.68,
    'USD': 10.70
}
base_year = 2024
discount_rate = 0.05
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = True
use_h2 = True
h2_initial = 150_000
biogas_limit = 500
load_target = 19 #TWh

In [3]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
section = None
cutout, selection, index = load_weather(geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-solar.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-onwind.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-offwind.nc").values.flatten()



In [4]:
# Build the network

network = build_network(index, resolution, geography, target_load, assumptions, capacity_factor_solar, capacity_factor_onwind, capacity_factor_offwind, use_offwind, use_h2, h2_initial, biogas_limit)

In [5]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

generator_capacity = model.variables["Generator-p_nom"]
link_capacity = model.variables["Link-p_nom"]

## Add offwind constraint
if use_offwind:
    offwind_percentage = 0.5

    offwind_constraint = (1 - offwind_percentage) / offwind_percentage * generator_capacity.loc['offwind'] - generator_capacity.loc['onwind']
    model.add_constraints(offwind_constraint == 0, name="Offwind_constraint")

## Add battery charge/discharge ratio constraint
lhs = link_capacity.loc["Battery charge"] - network.links.at["Battery charge", "efficiency"] * link_capacity.loc["Battery discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 2/2 [00:00<00:00, 1329.41it/s]
INFO:linopy.io: Writing time: 0.44s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [4e-06, 3e+02]
  Cost   [8e+01, 2e+06]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 3e+03]
Presolving model
48185 rows, 42352 cols, 127034 nonzeros  0s
38072 rows, 32239 cols, 117368 nonzeros  0s
37992 rows, 32159 cols, 117724 nonzeros  0s

Solving MIP model with:
   37992 rows
   32159 cols (0 binary, 5 integer, 0 implied int., 32154 continuous)
   117724 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.2s
         0       0         0   0.00%   29331394846.82  inf                  inf        0      0      1     58664    19.5s
 S       0       0         

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 46739 primals, 105142 duals
Objective: 4.38e+10
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance, Offwind_constraint were not assigned to the network.


('ok', 'optimal')

In [6]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator backstop       1889.269198                 0.0  1.526542e+05   
          biogas          500.000000                 0.0  1.749434e+05   
          offwind        9307.902857                 0.0  1.845578e+06   
          onwind         9307.902857                 0.0  1.633430e+07   
          solar          2456.717340                 0.0  2.775088e+06   
Link      AC             4216.773002                 0.0  1.773871e+07   
          biogas          500.000000                 0.0  1.749434e+05   
          h2             3000.000000                 0.0  3.601440e+06   
          li-ion          752.389766                 0.0  5.679446e+05   
          mixedgas       2100.000000                 0.0  1.134031e+06   
Load      -                 0.000000                 0.0  0.000000e+00   
Store     h2           104558.687987                 0.0  1.800720e+06   
          li-ion         5313.349197                 0.0  5.921233e+05   

                      Withdrawal      Dispatch  Transmission  Capacity Factor  \
Generator backstop  0.000000e+00  1.526542e+05  0.000000e+00         0.009224   
          biogas    0.000000e+00  1.749434e+05  0.000000e+00         0.039941   
          offwind   0.000000e+00  1.845578e+06  0.000000e+00         0.022635   
          onwind    0.000000e+00  1.633430e+07  0.000000e+00         0.200329   
          solar     0.000000e+00  2.775088e+06  0.000000e+00         0.128949   
Link      AC        1.776397e+07 -2.526002e+04  1.714537e+07         0.480901   
          biogas    1.749434e+05  0.000000e+00  1.749434e+05         0.039941   
          h2        4.991708e+06 -1.390268e+06  0.000000e+00         0.189943   
          li-ion    5.921233e+05 -2.417876e+04  0.000000e+00         0.089839   
          mixedgas  1.975663e+06 -8.416326e+05  1.975663e+06         0.107396   
Load      -         1.900000e+07 -1.900000e+07  0.000000e+00              NaN   
Store     h2        1.800720e+06 -4.583853e-10  0.000000e+00         0.335425   
          li-ion    5.933427e+05 -1.219382e+03  0.000000e+00         0.327501   

                     Curtailment  Capital Expenditure  \
Generator backstop  0.000000e+00         0.000000e+00   
          biogas    0.000000e+00         0.000000e+00   
          offwind   3.378513e+07         2.135240e+10   
          onwind    1.487334e+07         1.254669e+10   
          solar     1.959676e+02         1.276083e+09   
Link      AC        0.000000e+00         1.539972e+08   
          biogas    0.000000e+00         0.000000e+00   
          h2        0.000000e+00         9.820050e+08   
          li-ion    0.000000e+00         0.000000e+00   
          mixedgas  0.000000e+00         2.796449e+09   
Load      -         0.000000e+00         0.000000e+00   
Store     h2        0.000000e+00         1.110009e+09   
          li-ion    0.000000e+00         1.023309e+09   

                    Operational Expenditure  Revenue  Market Value  
Generator backstop             1.604701e+09      0.0           NaN  
          biogas               2.235322e+08      0.0           NaN  
          offwind              1.067390e+08      0.0           NaN  
          onwind               4.700941e+08      0.0           NaN  
          solar                0.000000e+00      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          biogas               0.000000e+00      0.0           NaN  
          h2                   0.000000e+00      0.0           NaN  
          li-ion               0.000000e+00      0.0           NaN  
          mixedgas             1.271448e+08      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     h2                   0.000000e+00      0.0           NaN  
          li-ion              -3.420793e+04      0.0           NaN